In [ ]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

mpl.rcParams.update({"font.size": 16})

from simpeg import maps
import simpeg.electromagnetics.time_domain as tdem

stepoff_waveform = tdem.sources.StepOffWaveform()

eps = 1e-6
ramp_on = np.r_[-0.004, -0.004 + eps]
ramp_off = np.r_[-eps, 0.0]
rectangular_waveform = tdem.sources.TrapezoidWaveform(
    ramp_on=ramp_on, ramp_off=ramp_off
)


def custom_waveform(t, tmax):
    out = np.cos(0.5 * np.pi * (t - tmax) / (tmax + 0.02))
    out[t >= tmax] = 1 + (t[t >= tmax] - tmax) / tmax
    return out


waveform_times = np.r_[np.linspace(-0.02, -0.011, 10), -np.logspace(-2, -6, 61), 0.0]
waveform_current = custom_waveform(waveform_times, -0.0055)
general_waveform = tdem.sources.PiecewiseLinearWaveform(
    times=waveform_times, currents=waveform_current
)

receiver_location = np.array([0.0, 0.0, 0.0])
receiver_orientation = "z"  # "x", "y" or "z"
times = np.logspace(-4, -1, 41)  # time channels

receiver_list = [
    tdem.receivers.PointMagneticFluxTimeDerivative(
        receiver_location, times, orientation=receiver_orientation
    )
]

source_location = np.array([0.0, 0.0, 0.0])
source_radius = 10.0
current_amplitude = 1.0

source_list = []

source_list.append(
    tdem.sources.CircularLoop(
        receiver_list=receiver_list,
        location=source_location,
        waveform=rectangular_waveform,
        radius=source_radius,
        current=current_amplitude,
    )
)

# Survey
survey = tdem.Survey(source_list)
# Layer thicknesses
thicknesses = np.array([40.0, 40.0])
n_layer = len(thicknesses) + 1

# half-space physical properties
sigma = 1e-2
eta = 0.5
tau = 0.01
c = 0.5
chi = 0.0

# physical property models
sigma_model = sigma * np.ones(n_layer)
eta_model = eta * np.ones(n_layer)
tau_model = tau * np.ones(n_layer)
c_model = c * np.ones(n_layer)
mu0 = 4 * np.pi * 1e-7
mu_model = mu0 * (1 + chi) * np.ones(n_layer)

# Define a mapping for conductivities
model_mapping = maps.IdentityMap(nP=n_layer)

# Define the simulation
simulation = tdem.Simulation1DLayered(
    survey=survey, thicknesses=thicknesses, sigmaMap=model_mapping, mu=mu_model
)

# Predict data for a given model
dpred = simulation.dpred(sigma_model)

fig = plt.figure(figsize=(8, 8))
d = np.reshape(dpred, (len(source_list), len(times))).T

<Figure size 800x800 with 0 Axes>